In [1]:
import datetime
from ib_insync import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

### Connect to IB

In [3]:
ib = connect_to_IB(clientid=40)
ac = initialize_db('db')

In [3]:
ac.list_libraries()

['general', 'pnl', 'portfolio', 'spx500', 'stocks']

In [4]:
# ac.delete_library('portfolio')

In [4]:
# pm = PortfolioManager(ib)
# Library(Arctic(config=LMDB(path=c:\Users\Johannes.Lange\pythonscripts\ATS\data_and_research\data_and_research\db)), path=portfolio, storage=lmdb_storage)
pm = PortfolioManager(ib,arctic=ac)


In [5]:
df = pm.get_ib_positions_for_gui()
df

0     37.0
1    887.0
2      1.0
3   -941.0
Name: position, dtype: float64
          timestamp symbol asset class  position   % of nav  averageCost  \
0  2024-05-30 15:27   AAPL         STK      37.0   3.184253   180.260541   
1  2024-05-30 15:27    CNR         STK    -699.0 -38.925099   174.287153   
2  2024-05-30 15:27    SIE         STK       1.0   0.085888   183.500000   

   marketPrice     pnl % strategy  \
0   190.460007  5.658180            
1   168.798004  3.149486            
2   175.487991 -4.366217            

                                            contract  ... close_dt deleted  \
0  Stock(conId=265598, symbol='AAPL', right='0', ...  ...            False   
1  Stock(conId=4458894, symbol='CNR', right='0', ...  ...            False   
2  Stock(conId=14217, symbol='SIE', right='0', pr...  ...            False   

  delete_dt  marketValue unrealizedPNL  currency  realizedPNL    account  \
0                7047.02        377.38       USD          0.0  DU3180649   
1     

,symbol,asset class,position,% of nav,currency,marketPrice,averageCost,pnl %,strategy
5,VOW,STK,765.0,49.735457,EUR,132.839127,140.970450,-5.768105,
4,VOW,STK,-941.0,-64.213458,EUR,132.839127,141.000179,5.787973,
1,CNR,STK,887.0,49.468440,CAD,168.798004,174.740231,-3.400606,
2,CNR,STK,-1586.0,-88.319326,CAD,168.798004,174.540546,3.290090,
0,AAPL,STK,37.0,3.200370,USD,190.460007,180.260541,5.658180,
3,SIE,STK,1.0,0.086182,EUR,175.487991,183.500000,-4.366217,


In [11]:
position_to_merge = ('VOW','',765)
target_position = ('VOW','',-941)

In [21]:
df_ac = pm.portfolio_library.read(pm.account_id).data

df_ac_active = df_ac[df_ac['deleted'] != True].copy()
latest_active_entries = df_ac_active.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class','position']).last().reset_index()
latest_active_entries

,symbol,strategy,asset class,position,timestamp,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,AAPL,,STK,37.0,2024-05-30 15:27,3.200370,180.260541,190.460007,5.658180,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,7047.020248,10.199466,USD,0.0,DU3180649,6539.153418,1.086015
1,CNR,,STK,-1586.0,2024-05-30 15:27,-88.319326,174.540546,168.798004,3.290090,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-267713.634582,9107.670660,CAD,0.0,DU3180649,-180458.392595,1.483520
2,CNR,,STK,887.0,2024-05-30 15:27,49.468440,174.740231,168.798004,-3.400606,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,149723.829681,-5.942227,CAD,0.0,DU3180649,101076.350426,1.482320
3,SIE,,STK,1.0,2024-05-30 15:27,0.086182,183.500000,175.487991,-4.366217,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,175.487991,-8.012009,EUR,0.0,DU3180649,176.092255,1.000000
4,VOW,,STK,-941.0,2024-05-30 15:27,-64.213458,141.000179,132.839127,5.787973,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-125001.618131,-8.161053,EUR,0.0,DU3180649,-131204.096924,1.000000
5,VOW,,STK,765.0,2024-05-30 15:27,49.735457,140.970450,132.839127,-5.768105,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,101621.931849,-6220.462401,EUR,0.0,DU3180649,101621.931849,1.000000


In [40]:
row_to_merge = latest_active_entries[(latest_active_entries['symbol'] == position_to_merge[0]) 
                & (latest_active_entries['strategy'] == position_to_merge[1]) 
                & (latest_active_entries['position'] == position_to_merge[2])].reset_index(drop=True)
target_row = latest_active_entries[(latest_active_entries['symbol'] == target_position[0]) 
                & (latest_active_entries['strategy'] == target_position[1]) 
                & (latest_active_entries['position'] == target_position[2])].reset_index(drop=True)
target_row

,symbol,strategy,asset class,position,timestamp,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,VOW,,STK,-941.0,2024-05-30 15:27,-64.213458,141.000179,132.839127,5.787973,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-125001.618131,-8.161053,EUR,0.0,DU3180649,-131204.096924,1.0


In [64]:
# if not target_row.empty:
#     print(True)


# target_row = target_row.iloc[0]
agg_df = target_row.copy()
agg_df.loc[0,'position'] += row_to_merge.loc[0,'position']
agg_df.loc[0,'timestamp'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
agg_df.loc[0,'marketValue_base'] += row_to_merge.loc[0,'marketValue_base']
agg_df.loc[0,'% of nav'] = ( agg_df.loc[0,'marketValue_base'] / pm.total_equity ) * 100
agg_df.loc[0,'averageCost'] = ( (row_to_merge.loc[0,'averageCost'] * row_to_merge.loc[0,'position']) + 
                                (target_row.loc[0,'averageCost'] * target_row.loc[0,'position']) ) / agg_df.loc[0,'position']
agg_df.loc[0,'pnl %'] = ( (agg_df.loc[0,'marketPrice'] - agg_df.loc[0,'averageCost']) * agg_df.loc[0,'position'] )/ agg_df.loc[0,'marketValue_base']
# skip contract
agg_df.loc[0,'trade'] = [agg_df.loc[0,'trade'], row_to_merge.loc[0,'trade']]

ValueError: Must have equal len keys and value when setting with an iterable

In [57]:
agg_df.loc[:,'pnl %':]

,pnl %,contract,trade,open_dt,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,-0.049323,"Stock(conId=14232, symbol='VOW', right='0', pr...",,2024-05-29,,False,,-125001.618131,-8.161053,EUR,0.0,DU3180649,-29582.165075,1.0


In [63]:
agg_df.contract[0]

"Stock(conId=14232, symbol='VOW', right='0', primaryExchange='IBIS', currency='EUR', localSymbol='VOW', tradingClass='XETRA')"

In [ ]:
trade_dict = {'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
            'symbol': trade.contract.symbol,
            'asset class': trade.contract.secType,
            'position': trade.order.totalQuantity,
            '% of nav': 0.0, # to be calculated
            'averageCost': trade.orderStatus.avgFillPrice,
            'marketPrice': trade.orderStatus.lastFillPrice,
            'pnl %': 0.0, # to be calculated
            'strategy': strategy_symbol,
            'contract': str(trade.contract),
            'trade': str(trade),
            'open_dt':datetime.date.today().isoformat(),
            'close_dt': '',
            'deleted': False,
            'delete_dt': '',
            'marketValue': 0.0, # to be calculated
            'unrealizedPNL': 0.0, # to be calculated
            'currency':trade.contract.currency,
            'realizedPNL': 0.0, # to be calculated
            'account': trade.fills[0].execution.acctNumber,
            'marketValue_base': 0.0, # to be calculated
            'fx_rate': 0.0}
        except Exception as e:
            print(f"Error processing trade: {e}")


In [24]:

# print("print from match_ib_positions_with_arcticdb")
if pm.account_id in pm.portfolio_library.list_symbols():
    df_ac = pm.portfolio_library.read(f'{pm.account_id}').data
    # Filter out deleted entries before comparing
    df_ac_active = df_ac[df_ac['deleted'] != True].copy()
    latest_positions_in_ac = df_ac_active.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class','position']).last().reset_index()
    print(latest_positions_in_ac['position'])
else:
    print(f"Msg from function 'match_ib_positions_with_arcticdb': No Symbol {pm.account_id} in library 'portfolio'")
    df_ib = pm.get_positions_from_ib()

df_ib = pm.get_positions_from_ib()
df_merged = pd.DataFrame()

# Iterate through the positions obtained from Interactive Brokers
for index, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']

    # Filter the ArcticDB DataFrame for entries with the same symbol and asset class
    strategy_entries_in_ac = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]
    sum_of_position_entries = strategy_entries_in_ac['position'].sum()
    print(strategy_entries_in_ac)
    if strategy_entries_in_ac.empty: # no database entry, add position
        df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

    elif len(strategy_entries_in_ac) == 1:
        if total_position == sum_of_position_entries:
            # the positions match, we should only update fields relevant for marketdata in ac
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
        else:
            #update marketdata relevant fields for the existing db entry
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # handle the residual and concat to df_merged
            residual = pm.handle_residual(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, residual], ignore_index=True)

    elif len(strategy_entries_in_ac) > 1:
        if total_position == sum_of_position_entries:
            print("len(strategy_entries_in_ac) > 1 and total_position == sum_of_position_entries")
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
        else:
            print("len(strategy_entries_in_ac) > 1 and total_position != sum_of_position_entries")
            #update marketdata relevant fields for the existing db entry
            strategy_entries_in_ac = pm.update_data(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # handle the residual and concat to df_merged
            residual = pm.handle_residual(strategy_entries_in_ac,row)
            df_merged = pd.concat([df_merged, residual], ignore_index=True)
    

0      37.0
1   -1586.0
2     887.0
3       1.0
4    -941.0
5     765.0
Name: position, dtype: float64
  symbol strategy asset class  position         timestamp  % of nav  \
0   AAPL                  STK      37.0  2024-05-30 15:27   3.20037   

   averageCost  marketPrice    pnl %  \
0   180.260541   190.460007  5.65818   

                                            contract  ... close_dt deleted  \
0  Stock(conId=265598, symbol='AAPL', right='0', ...  ...            False   

  delete_dt  marketValue unrealizedPNL  currency  realizedPNL    account  \
0            7047.020248     10.199466       USD          0.0  DU3180649   

   marketValue_base   fx_rate  
0       6539.153418  1.086015  

[1 rows x 22 columns]
  symbol strategy asset class  position         timestamp   % of nav  \
1    CNR                  STK   -1586.0  2024-05-30 15:27 -88.319326   
2    CNR                  STK     887.0  2024-05-30 15:27  49.468440   

   averageCost  marketPrice     pnl %  \
1   174.540546   1

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
df_merged

In [8]:
df = pm.portfolio_library.read('test').data


In [13]:
for col in df.columns:
    print(f"{col}: {type(df[col][0])}")

timestamp: <class 'str'>
symbol: <class 'str'>
asset class: <class 'str'>
position: <class 'numpy.float64'>
% of nav: <class 'numpy.float64'>
averageCost: <class 'numpy.float64'>
marketPrice: <class 'numpy.float64'>
pnl %: <class 'numpy.float64'>
strategy: <class 'str'>
contract: <class 'str'>
trade: <class 'str'>
open_dt: <class 'str'>
close_dt: <class 'str'>
deleted: <class 'numpy.bool_'>
delete_dt: <class 'str'>
marketValue: <class 'numpy.float64'>
unrealizedPNL: <class 'numpy.float64'>
currency: <class 'str'>
realizedPNL: <class 'numpy.float64'>
account: <class 'str'>
marketValue_base: <class 'numpy.float64'>
fx_rate: <class 'numpy.float64'>


In [5]:
df = pm.get_positions_from_ib()
if df.empty:
    print(" empty")

In [7]:
ac.get_library('portfolio').read('test').data.tail()

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
33,CNR,,STK,2024-05-29 11:27,887.0,51.490026,174.740231,169.473999,-3.013749,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,150323.437113,-5.266232,CAD,0.00,DU3180649,104380.230712,1.48183
34,CNR,,STK,2024-05-29 11:27,-1586.0,-89.489351,174.540546,169.473999,2.902791,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-268785.762414,8035.542828,CAD,0.00,DU3180649,-181412.203056,1.48163
35,SIE,,STK,2024-05-29 11:27,1.0,0.089241,183.500000,176.932007,-3.579288,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,176.932007,-6.567993,EUR,0.00,DU3180649,180.907990,1.00000
36,VOW,,STK,2024-05-29 11:27,765.0,53.865656,140.970450,139.963074,-0.714601,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,107071.751419,-1.007376,EUR,-3230.09,DU3180649,109196.092541,1.00000
37,VOW,,STK,2024-05-29 11:27,-941.0,-64.969265,141.000179,139.963074,0.735535,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-131705.252399,975.916251,EUR,0.00,DU3180649,-131705.252399,1.00000


In [13]:
ac

Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))

In [15]:
df = pm.get_ib_positions_for_gui()
df

0     37.0
1   -699.0
2      1.0
3   -176.0
Name: position, dtype: float64
          timestamp symbol asset class  position   % of nav  averageCost  \
0  2024-05-29 12:21   AAPL         STK      37.0   3.191450   180.260541   
1  2024-05-29 12:21    CNR         STK    -699.0 -39.411599   174.287153   
2  2024-05-29 12:21    SIE         STK       1.0   0.087447   183.500000   

   marketPrice     pnl % strategy  \
0   189.779999  5.280944            
1   169.473999  2.761623            
2   177.360001 -3.346049            

                                            contract  ... close_dt deleted  \
0  Stock(conId=265598, symbol='AAPL', right='0', ...  ...            False   
1  Stock(conId=4458894, symbol='CNR', right='0', ...  ...            False   
2  Stock(conId=14217, symbol='SIE', right='0', pr...  ...            False   

  delete_dt  marketValue unrealizedPNL  currency  realizedPNL    account  \
0                7021.86        352.22       USD          0.0  DU3180649   
1     

StreamDescriptorMismatch: The columns (names and types) in the argument are not identical to that of the existing version: APPEND 
(Showing only the mismatch. Full col list saved in the `last_mismatch_msg` attribute of the lib instance.
'-' marks columns missing from the argument, '+' for unexpected.)
+FD<name=symbol, type=TD<type=UTF_DYNAMIC64, dim=0>>
+FD<name=strategy, type=TD<type=UTF_DYNAMIC64, dim=0>>
+FD<name=asset class, type=TD<type=UTF_DYNAMIC64, dim=0>>
-FD<name=symbol, type=TD<type=UTF_DYNAMIC64, dim=0>>
-FD<name=asset class, type=TD<type=UTF_DYNAMIC64, dim=0>>
-FD<name=strategy, type=TD<type=UTF_DYNAMIC64, dim=0>>

In [102]:
df_ac = ac.get_library('portfolio').read('positions').data
# Filter out deleted entries before comparing
df_ac_active = df_ac[df_ac['deleted'] != True].copy()
df_ac_active
#latest_positions_in_ac = df_ac_active.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()

NoSuchVersionException: E_NO_SUCH_VERSION read_dataframe_version: version matching query 'latest' not found for symbol 'positions'

In [109]:
symbol = "XLU"
asset_class = 'Put 63.0 20240419'
position = -1
strategy = ''


def delete_symbol(symbol, asset_class, position, strategy):
    # Load the library and read the existing DataFrame
    lib = ac.get_library('portfolio')
    df = lib.read('positions').data

    # Ensure 'deleted' and 'delete_dt' columns exist
    if 'deleted' not in df.columns:
        df['deleted'] = False
    if 'delete_dt' not in df.columns:
        df['delete_dt'] = None

    # Filter for matching entries that have not been previously marked as deleted
    filter_condition = (df['symbol'] == symbol) & \
                       (df['asset class'] == asset_class) & \
                       (df['position'] == float(position)) & \
                       (df['strategy'] == strategy) & \
                       (df['deleted'] == False)

    # Mark the filtered entries as deleted and add the current timestamp
    df.loc[filter_condition, 'deleted'] = True
    df.loc[filter_condition, 'delete_dt'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Convert to string

    # Save the updated DataFrame back to ArcticDB
    if 'positions' in lib.list_symbols():
        df = normalize_columns(df)
        lib.write('positions', df, prune_previous_versions=True)
        print(f"Deleted positions for {symbol} {asset_class} with position {position}")
    else:
        print("Positions symbol not found in library.")
            
def normalize_columns(df):
    df = df.copy()
    df['contract'] = df['contract'].astype(str)
    df['trade'] = df['trade'].astype(str)
    return df

# df = ac.get_library('portfolio').read('positions').data
# df

In [114]:
lib = ac.get_library('portfolio')
df = lib.read('positions').data
filter_condition = (df['symbol'] == symbol) & \
                       (df['asset class'] == asset_class) & \
                       (df['position'] == float(position)) & \
                       (df['strategy'] == strategy) & \
                       (df['deleted'] == False)

df[filter_condition]

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
63,XLU,,Put 63.0 20240419,2024-03-24 13:20,-1.0,-0.050033,86.1997,0.568817,34.011684,"Option(conId=652393383, symbol='XLU', lastTrad...",...,None,False,,-56.88,29.32,USD,0.0,U7706434,-52.625375,1.080847


In [115]:
delete_symbol(symbol,asset_class,position,strategy)
df = ac.get_library('portfolio').read('positions').data
df

[2024-03-24 13:24:13.744] [arcticdb] [info] Column close_dt does not have non null elements.


Deleted positions for XLU Put 63.0 20240419 with position -1


,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,test,STK,2024-03-23 23:43,30.0,1.904720,76.181877,72.180000,-5.253056,"Stock(conId=166090175, symbol='BABA', right='0...",...,None,True,2024-03-23 23:45:19,2165.400000,-120.060000,USD,0.0,U7706434,2003.428051,1.080847
1,EWT,,STK,2024-03-23 23:43,100.0,4.281969,44.256829,48.680000,9.994325,"Stock(conId=253190576, symbol='EWT', right='0'...",...,None,False,,4868.000000,442.320000,USD,0.0,U7706434,4503.873535,1.080847
2,EWT,,Call 50.0 20240419,2024-03-23 23:43,-1.0,-0.046540,48.960000,0.529119,-8.071691,"Option(conId=684551409, symbol='EWT', lastTrad...",...,None,False,,-52.910000,-3.950000,USD,0.0,U7706434,-48.952331,1.080847
3,IAU,,STK,2024-03-23 23:43,300.0,10.806104,38.454255,40.950100,6.490427,"Stock(conId=474829650, symbol='IAU', right='0'...",...,None,False,,12285.030000,748.750000,USD,0.0,U7706434,11366.109591,1.080847
4,IAU,,Call 39.0 20240419,2024-03-23 23:43,-1.0,-0.188853,44.950000,2.146970,-377.635106,"Option(conId=648748170, symbol='IAU', lastTrad...",...,None,False,,-214.700000,-169.750000,USD,0.0,U7706434,-198.640437,1.080847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,TLT,,Put 92.0 20240419,2024-03-24 13:20,-2.0,-0.000885,96.159600,0.502863,47.705393,"Option(conId=626935162, symbol='TLT', lastTrad...",...,None,False,,-1.005726,-95.656737,USD,0.0,U7706434,-0.930497,1.080847
60,TLT,,Put 94.0 20240419,2024-03-24 13:20,-2.0,-0.002187,113.309450,1.243414,-9.736152,"Option(conId=626935337, symbol='TLT', lastTrad...",...,None,False,,-2.486829,-112.066036,USD,0.0,U7706434,-2.300814,1.080847
61,TLT,,Call 95.0 20240419,2024-03-24 13:20,-1.0,-0.000780,68.949800,0.886399,-28.557160,"Option(conId=626933014, symbol='TLT', lastTrad...",...,None,False,,-0.886399,-68.063401,USD,0.0,U7706434,-0.820096,1.080847
62,TLT,,Call 97.0 20240426,2024-03-24 13:20,-2.0,-0.000807,46.798750,0.458580,2.010235,"Option(conId=689139606, symbol='TLT', lastTrad...",...,None,False,,-0.917160,-46.340170,USD,0.0,U7706434,-0.848556,1.080847


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederherg

In [60]:
latest_positions_in_ac = df_ac_active.groupby(['symbol', 'strategy', 'asset class']).apply(lambda x: x.sort_values(by='timestamp', ascending=False).head(1)).reset_index(drop=True)
latest_positions_in_ac

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-03-23 18:23,30.0,1.904720,76.181877,72.180000,-5.253056,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-03-23,None,False,2165.400009,-4.001876,USD,0.0,U7706434,2003.428059,1.080847
1,EWT,,Call 50.0 20240419,2024-03-23 18:23,-1.0,-0.000465,48.960000,0.529119,-8.071691,"Option(conId=684551409, symbol='EWT', lastTrad...",...,2024-03-23,None,False,-0.529119,-48.430881,USD,0.0,U7706434,-0.489541,1.080847
2,EWT,,STK,2024-03-23 18:23,100.0,4.281969,44.256829,48.680000,9.994325,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-03-23,None,False,4868.000030,4.423171,USD,0.0,U7706434,4503.873563,1.080847
3,IAU,,Call 39.0 20240419,2024-03-23 18:23,-1.0,-0.001889,44.950000,2.146970,-377.635106,"Option(conId=648748170, symbol='IAU', lastTrad...",...,2024-03-23,None,False,-2.146970,-42.803030,USD,0.0,U7706434,-1.986376,1.080847
4,IAU,,STK,2024-03-23 18:23,300.0,10.806104,38.454255,40.950100,6.490427,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-03-23,None,False,12285.029985,2.495845,USD,0.0,U7706434,11366.109578,1.080847
5,INDA,,STK,2024-03-23 18:23,100.0,4.440467,49.712499,50.481899,1.547700,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-03-23,None,False,5048.189925,0.769400,USD,0.0,U7706434,4670.585251,1.080847
6,IUSQ,,STK,2024-03-23 18:23,100.0,7.171358,65.602785,75.430000,14.979875,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-03-23,None,False,7543.000030,9.827215,EUR,0.0,U7706434,7543.000030,1.000000
7,MTUM,,Put 175.0 20240419,2024-03-23 18:23,1.0,0.000517,154.996900,0.587429,-62.100581,"Option(conId=684706174, symbol='MTUM', lastTra...",...,2024-03-23,None,False,0.587429,-154.409471,USD,0.0,U7706434,0.543490,1.080847
8,PRX,,STK,2024-03-23 18:23,250.0,6.593314,26.561554,27.740000,4.436658,"Stock(conId=382625193, symbol='PRX', right='0'...",...,2024-03-23,None,False,6934.999937,1.178445,EUR,0.0,U7706434,6934.999937,1.000000
9,QQQ,,STK,2024-03-23 18:23,10.0,3.920447,447.040500,445.700012,-0.299858,"Stock(conId=320227571, symbol='QQQ', right='0'...",...,2024-03-23,None,False,4457.000122,-1.340488,USD,0.0,U7706434,4123.616453,1.080847


In [49]:
lib = ac.get_library('portfolio')
positions_df = lib.read('positions').data
positions_df

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-03-23 12:26,30.0,1.904720,76.181877,72.180000,-5.253056,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-03-23,None,False,2165.400000,-120.060000,USD,0.0,U7706434,2003.428051,1.080847
1,EWT,,STK,2024-03-23 12:26,100.0,4.281969,44.256829,48.680000,9.994325,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-03-23,None,False,4868.000000,442.320000,USD,0.0,U7706434,4503.873535,1.080847
2,EWT,,Call 50.0 20240419,2024-03-23 12:26,-1.0,-0.046540,48.960000,0.529119,-8.071691,"Option(conId=684551409, symbol='EWT', lastTrad...",...,2024-03-23,None,False,-52.910000,-3.950000,USD,0.0,U7706434,-48.952331,1.080847
3,IAU,,STK,2024-03-23 12:26,300.0,10.806104,38.454255,40.950100,6.490427,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-03-23,None,False,12285.030000,748.750000,USD,0.0,U7706434,11366.109591,1.080847
4,IAU,,Call 39.0 20240419,2024-03-23 12:26,-1.0,-0.188853,44.950000,2.146970,-377.635106,"Option(conId=648748170, symbol='IAU', lastTrad...",...,2024-03-23,None,False,-214.700000,-169.750000,USD,0.0,U7706434,-198.640437,1.080847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,TLT,,Put 94.0 20240419,2024-03-23 15:10,-2.0,-0.002187,113.309450,1.243414,-9.736152,"Option(conId=626935337, symbol='TLT', lastTrad...",...,2024-03-23,None,False,-2.486829,-112.066036,USD,0.0,U7706434,-2.300814,1.080847
95,TLT,,Call 95.0 20240419,2024-03-23 15:10,-1.0,-0.000780,68.949800,0.886399,-28.557160,"Option(conId=626933014, symbol='TLT', lastTrad...",...,2024-03-23,None,False,-0.886399,-68.063401,USD,0.0,U7706434,-0.820096,1.080847
96,TLT,,Call 97.0 20240426,2024-03-23 15:10,-2.0,-0.000807,46.798750,0.458580,2.010235,"Option(conId=689139606, symbol='TLT', lastTrad...",...,2024-03-23,None,False,-0.917160,-46.340170,USD,0.0,U7706434,-0.848556,1.080847
97,XLU,test,Put 63.0 20240419,2024-03-23 15:10,-1.0,-0.000500,86.199700,0.568817,34.011684,"Option(conId=652393383, symbol='XLU', lastTrad...",...,2024-03-23,None,False,-0.568817,-85.630883,USD,0.0,U7706434,-0.526270,1.080847


In [30]:
lib.delete('positions')

[2024-03-23 12:21:00.301] [arcticdb] [warning] Symbol: positions does not exist.


### Create an Instance of PortfolioManager

In [17]:
pm = PortfolioManager(ib)

#### Get the current positions from your portfolio by strategy as assigned in ArcticDB

In [29]:
# df = pm.match_ib_positions_with_arcticdb()
# df.tail()
pm.delete_symbol_from_portfolio_lib('positions')

[2024-03-23 12:20:30.514] [arcticdb] [warning] Symbol: positions does not exist.


In [ ]:
ac.get_library('portfolio').delete('positions')

Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: cashfarm; usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: cashfarm; usfarm.nj; eufarm; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren 

#### Functions
- to assign a position to a strategy
- to delete a position in ArcticDB 
(will be written again to acdb if still in portfolio, but should lose its strategy assignment )

In [27]:
def assign_strategy(df, idx, strategy):
    # Get the symbol from the row
    if idx in df.index:
        symbol = df.loc[idx,'symbol']
        asset_class = df.loc[idx,'asset class']
        position = df.loc[idx,'position']

        filter_cond = (df['symbol'] == symbol) & \
                  (df['asset class'] == asset_class) & \
                  (df['position'] == position)
        
        filtered_df = df.loc[filter_cond]

        if not filtered_df.empty:
            # Update the strategy for the filtered rows
            df.loc[filter_cond, 'strategy'] = strategy
        
        pm.library.write('positions', df, metadata={'updated': 'strategy assignment from gui'})

def delete_strategy(df,idx,strategy):
    
        
assign_strategy(df,8,'SVIX')

In [28]:
df

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-02-19 20:58,30.0,1.985980,76.181877,73.820000,-3.100313,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-19,None,False,2214.599991,-2.361877,USD,0.0,U7706434,2054.931791,NaN
1,EWT,,STK,2024-02-19 20:58,200.0,8.266396,44.256828,46.090000,4.142122,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-19,None,False,9218.000030,1.833172,USD,0.0,U7706434,8553.400789,NaN
2,IAU,,STK,2024-02-19 20:58,300.0,10.247381,38.454255,38.090099,-0.946983,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-02-19,None,False,11427.029805,-0.364155,USD,0.0,U7706434,10603.163965,NaN
3,INDA,,STK,2024-02-19 20:58,100.0,4.592693,49.712499,51.213901,3.020171,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-02-19,None,False,5121.390150,1.501402,USD,0.0,U7706434,4752.148232,NaN
4,IUSQ,,STK,2024-02-19 20:58,100.0,7.019294,65.602785,72.629997,10.711759,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-02-19,None,False,7262.999725,7.027212,EUR,0.0,U7706434,7262.999725,1.0
5,PRX,,STK,2024-02-19 20:58,250.0,6.937123,26.561554,28.711903,8.095717,"Stock(conId=382625193, symbol='PRX', right='0'...",...,2024-02-19,None,False,7177.975650,2.150348,EUR,0.0,U7706434,7177.975650,1.0
6,TLT,,STK,2024-02-19 20:58,300.0,25.017103,97.181951,92.989998,-4.313510,"Stock(conId=15547841, symbol='TLT', right='0',...",...,2024-02-19,None,False,27896.999355,-4.191954,USD,0.0,U7706434,25885.681873,NaN
7,TLT,,Put 93.0 20240223,2024-02-19 20:58,-1.0,-0.000516,93.959600,0.575791,38.719290,"Option(conId=676006202, symbol='TLT', lastTrad...",...,2024-02-19,None,False,-0.575791,-93.383809,USD,0.0,U7706434,-0.534278,NaN
8,VXM,SVIX,VXMJ4 20240417,2024-02-19 20:58,-3.0,-0.042884,1615.525000,15.940211,1.331076,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-19,None,False,-47.820634,-1599.584789,USD,0.0,U7706434,-44.372862,NaN


#### Delete 'positions' from library portfolio

In [51]:
#pm.delete_symbol_from_portfolio_lib('positions')
ac.get_library('portfolio').delete('positions')

In [ ]:
#pm.library.read('positions').data
ac.get_library('portfolio').read('positions').data

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,...,open_dt,close_dt,deleted,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,BABA,,STK,2024-02-19 22:44,30.0,1.986228,76.181877,73.820000,-3.100313,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-19,None,False,2214.600000,-70.860000,USD,0.0,U7706434,2054.931799,1.0777
0,EWT,,STK,2024-02-19 22:44,200.0,8.267428,44.256828,46.090000,4.142122,"Stock(conId=253190576, symbol='EWT', right='0'...",...,2024-02-19,None,False,9218.000000,366.630000,USD,0.0,U7706434,8553.400761,1.0777
0,IAU,,STK,2024-02-19 22:44,300.0,10.248660,38.454255,38.090099,-0.946983,"Stock(conId=474829650, symbol='IAU', right='0'...",...,2024-02-19,None,False,11427.030000,-109.250000,USD,0.0,U7706434,10603.164146,1.0777
0,INDA,,STK,2024-02-19 22:44,100.0,4.593266,49.712499,51.213901,3.020171,"Stock(conId=101484826, symbol='INDA', right='0...",...,2024-02-19,None,False,5121.390000,150.140000,USD,0.0,U7706434,4752.148093,1.0777
0,IUSQ,,STK,2024-02-19 22:44,100.0,7.020170,65.602785,72.629997,10.711759,"Stock(conId=239363241, symbol='IUSQ', right='0...",...,2024-02-19,None,False,7263.000000,702.720000,EUR,0.0,U7706434,7263.000000,1.0000
0,PRX,,STK,2024-02-19 22:44,250.0,6.937993,26.561554,28.711903,8.095717,"Stock(conId=382625193, symbol='PRX', right='0'...",...,2024-02-19,None,False,7177.980000,537.590000,EUR,0.0,U7706434,7177.980000,1.0000
0,TLT,,STK,2024-02-19 22:44,300.0,25.020225,97.181951,92.989998,-4.313510,"Stock(conId=15547841, symbol='TLT', right='0',...",...,2024-02-19,None,False,27897.000000,-1257.590000,USD,0.0,U7706434,25885.682472,1.0777
0,TLT,,Put 93.0 20240223,2024-02-19 22:44,-1.0,-0.051642,93.959600,0.575791,38.719290,"Option(conId=676006202, symbol='TLT', lastTrad...",...,2024-02-19,None,False,-57.580000,36.380000,USD,0.0,U7706434,-53.428598,1.0777
0,VXM,SVIX,VXMJ4 20240417,2024-02-19 22:44,-3.0,-4.288928,1615.525000,15.940211,1.331076,"Future(conId=661277712, symbol='VXM', lastTrad...",...,2024-02-19,None,False,-4782.060000,64.510000,USD,0.0,U7706434,-4437.283103,1.0777
0,BABA,,STK,2024-02-19 22:44,30.0,1.986228,76.181877,73.820000,-3.100313,"Stock(conId=166090175, symbol='BABA', right='0...",...,2024-02-19,None,False,2214.599991,-2.361877,USD,0.0,U7706434,2054.931799,1.0777


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfuture.nj; usfarm.nj; eufarm; cashfarm; usopt; usfarm; euhmds; ushmds; secdefeu.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfuture.nj; usfarm.nj; eufarm; cashfarm; usopt; usfarm; euhmds; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME%